# Strands マルチエージェントシステムと AgentCore Memory Tool（短期メモリ）

## はじめに

このノートブックでは、AWS AgentCore Memory と Strands フレームワークを使用して**共有メモリを持つマルチエージェントシステム**を実装する方法を紹介します。これまでの例では単一エージェントのメモリに焦点を当てていましたが、このノートブックでは複数の専門エージェントが共通のメモリストアにアクセスしながら連携する方法を探ります。

## チュートリアルの詳細

| 情報               | 詳細                                                                            |
|:-------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話メモリ                                                                  |
| エージェントのユースケース | 旅行プランニングアシスタント                                                    |
| エージェントフレームワーク | Strands Agents                                                                  |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                     |
| チュートリアルコンポーネント | AgentCore 短期メモリ、Strands Agents、Tool によるメモリ取得                       |
| 難易度              | 初級                                                                            |


学習内容：

- 複数のエージェントがアクセスできる共有メモリリソースの設定方法
- 独自のメモリアクセスを持つツールとしての専門エージェントの作成
- 専門エージェントに委譲するコーディネーターエージェントの実装
- 複数のエージェントインタラクション間での会話コンテキストの維持

### シナリオのコンテキスト

この例では、以下を備えた**旅行プランニングシステム**を作成します：
1. 航空旅行に特化したフライト予約アシスタント
2. 宿泊施設に焦点を当てたホテル予約アシスタント
3. これらの専門エージェントに委譲するトラベルコーディネーター

このアプローチは、複雑なドメインをメモリストアを共有する専門エージェントに分割できることを示しています。

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件
- Python 3.10以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory の適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

環境のセットアップと共有メモリリソースの作成から始めましょう！

## ステップ 1: 環境のセットアップ
まず、このノートブックを動作させるために必要なすべてのライブラリをインポートし、クライアントを定義します。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

Amazon Bedrock モデルと AgentCore の適切な権限を持つリージョンとロールを定義します

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')
MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## ステップ 2: 共有メモリの作成
このセクションでは、専門エージェント間で共有されるメモリリソースを作成します。

In [ ]:
from bedrock_agentcore.memory import MemoryClient

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "TravelAgent_STM_%s" % datetime.now().strftime("%Y%m%d%H%M%S")
memory_id = None


In [ ]:
from botocore.exceptions import ClientError

try:
    print("メモリを作成中...")
    memory_name = memory_name

    # メモリリソースを作成
    memory = client.create_memory_and_wait(
        name=memory_name,                       # このメモリストアの一意の名前
        description="Travel Agent STM",         # 人間が読める説明
        strategies=[],                          # 短期メモリには特別なメモリ戦略なし
        event_expiry_days=7,                    # メモリは 7 日後に期限切れ
        max_wait=300,                           # メモリ作成を待つ最大時間（5 分）
        poll_interval=10                        # 10 秒ごとにステータスをチェック
    )

    # メモリ ID を抽出して表示
    memory_id = memory['id']
    print(f"メモリが正常に作成されました。ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # メモリが既に存在する場合は、その ID を取得
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"メモリは既に存在します。既存のメモリ ID を使用: {memory_id}")
except Exception as e:
    # メモリ作成中のエラーを処理
    print(f"❌ エラー: {e}")
    import traceback
    traceback.print_exc()

    # エラー時のクリーンアップ - 部分的に作成されたメモリを削除
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"メモリをクリーンアップしました: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"メモリのクリーンアップに失敗しました: {cleanup_error}")

### マルチエージェントシステムのための共有メモリの理解

作成したメモリリソースは、旅行プランニングシステムの共有知識ベースとして機能します。すべてのエージェントがこの共通メモリストアから読み書きを行い、以下を実現します：

1. **知識の一貫性**: すべてのエージェントが同じ情報で動作
2. **コンテキストの保持**: エージェント間の遷移を超えて会話履歴が維持される
3. **特化されたアクセス**: 各エージェントは独自の actor_id を持つが、session_id は共有

このアプローチにより、専門エージェントは完全な会話コンテキストの恩恵を受けながら、それぞれのドメインに集中できます。

## ステップ 3: Memory Hook Provider の作成

このステップでは、メモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。Hooks はエージェントの実行ライフサイクルの特定のポイントで実行される特別な関数です。作成するメモリ Hook は 2 つの主要な機能を果たします：

1. **メモリの取得**: ユーザーがメッセージを送信したときに、関連する過去の会話を自動的に取得
2. **メモリの保存**: エージェントが応答した後に新しい会話を保存

これにより、手動管理なしでシームレスなメモリ体験が実現されます。

In [ ]:
class ShortTermMemoryHook(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """エージェント開始時に最近の会話履歴を読み込む"""
        try:
            # エージェント状態からセッション情報を取得
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                logger.warning("エージェント状態に actor_id または session_id がありません")
                return
            
            # 最後の 5 つの会話ターンを取得
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=actor_id,
                session_id=session_id,
                k=5,
                branch_name="main"
            )
            
            if recent_turns:
                # コンテキスト用に会話履歴をフォーマット
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role'].lower()
                        content = message['content']['text']
                        context_messages.append(f"{role.title()}: {content}")
                
                context = "\n".join(context_messages)
                logger.info(f"メモリからのコンテキスト: {context}")
                
                # エージェントのシステムプロンプトにコンテキストを追加
                event.agent.system_prompt += f"\n\n最近の会話履歴:\n{context}\n\nこのコンテキストに基づいて自然に会話を続けてください。"
                
                logger.info(f"✅ {len(recent_turns)} 件の最近の会話ターンを読み込みました")
            else:
                logger.info("以前の会話履歴が見つかりません")
                
        except Exception as e:
            logger.error(f"会話履歴の読み込みに失敗しました: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """会話ターンをメモリに保存"""
        messages = event.agent.messages
        try:
            # エージェント状態からセッション情報を取得
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                logger.warning("エージェント状態に actor_id または session_id がありません")
                return
            
            self.memory_client.create_event(
                memory_id=self.memory_id,
                actor_id=actor_id,
                session_id=session_id,
                messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
            )
            
        except Exception as e:
            logger.error(f"メッセージの保存に失敗しました: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        # メモリ Hooks を登録
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## ステップ 4: Strands Agents によるマルチエージェントアーキテクチャの作成
このセクションでは、フライトとホテル予約の専門エージェントを作成します。両方がメモリリソースへの共有アクセスを持ちます。

In [ ]:
# 必要なコンポーネントをインポート
from strands import Agent, tool

In [ ]:
# 各専門エージェントに一意の actor ID を作成し、session ID は共有
flight_actor_id = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
hotel_actor_id = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
flight_namespace = f"travel/{flight_actor_id}/preferences"
hotel_namespace = f"travel/{hotel_actor_id}/preferences"

### メモリアクセスを持つ専門エージェントの作成

次に、専門エージェントのシステムプロンプトを定義します。各プロンプトには、エージェントが解析できる形式でメモリパラメータが含まれています：

In [ ]:
# ホテル予約スペシャリストのシステムプロンプト
HOTEL_BOOKING_PROMPT = f"""あなたはホテル予約アシスタントです。お客様がホテルを探したり、予約を行ったり、宿泊施設やアメニティに関する質問に答えるお手伝いをします。
空室状況、料金、予約手続きについて、親切で丁寧な対応で明確な情報を提供してください。"""

# フライト予約スペシャリストのシステムプロンプト
FLIGHT_BOOKING_PROMPT = f"""あなたはフライト予約アシスタントです。お客様がフライトを探したり、予約を行ったり、航空会社、路線、旅行ポリシーに関する質問に答えるお手伝いをします。
フライトの空席状況、料金、スケジュール、予約手続きについて、親切で丁寧な対応で明確な情報を提供してください。"""

### エージェントツールの実装
コーディネーターエージェントが使用できるツールとして専門エージェントを実装します：

In [ ]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    フライト予約クエリを処理して応答します。

    Args:
        query: 予約、スケジュール、航空会社、または旅行ポリシーに関するフライト関連の質問

    Returns:
        詳細なフライト情報、予約オプション、または旅行アドバイス
    """
    try:
        flight_memory_hooks = ShortTermMemoryHook(client, memory_id)
        
        flight_agent = Agent(
            hooks=[flight_memory_hooks],
            model=MODEL_ID,
            system_prompt=FLIGHT_BOOKING_PROMPT,
            state={"actor_id": flight_actor_id, "session_id": session_id}
        )

        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"フライト予約アシスタントでエラー: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    ホテル予約クエリを処理して応答します。

    Args:
        query: 宿泊施設、アメニティ、または予約に関するホテル関連の質問

    Returns:
        詳細なホテル情報、予約オプション、または宿泊アドバイス
    """
    try:
        hotel_memory_hooks = ShortTermMemoryHook(client, memory_id)

        hotel_booking_agent = Agent(
            hooks=[hotel_memory_hooks],
            model=MODEL_ID,
            system_prompt=HOTEL_BOOKING_PROMPT,
            state={"actor_id": hotel_actor_id, "session_id": session_id}
        )
        
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"ホテル予約アシスタントでエラー: {str(e)}"

### コーディネーターエージェントの作成

最後に、これらの専門ツール間を調整するメインの旅行プランニングエージェントを作成します：

In [ ]:
# コーディネーターエージェントのシステムプロンプト
TRAVEL_AGENT_SYSTEM_PROMPT = """
あなたは専門ツール間を調整する包括的な旅行プランニングアシスタントです：
- フライト関連のクエリ（予約、スケジュール、航空会社、路線）→ flight_booking_assistant ツールを使用
- ホテル関連のクエリ（宿泊施設、アメニティ、予約）→ hotel_booking_assistant ツールを使用
- 完全な旅行パッケージ → 必要に応じて両方のツールを使用して包括的な情報を提供
- 一般的な旅行アドバイスや簡単な旅行の質問 → 直接回答

各エージェントは、ユーザーが履歴データについて質問した場合に備えて、独自のメモリを持っています。
複雑な旅行リクエストを処理する際は、両方のツールからの情報を調整して、まとまりのある旅行プランを作成してください。
複数のソースからの情報を提示する際は、明確に整理して提供してください。
1 ターンあたり最大 2 つの質問をしてください。メッセージは短くし、お客様を圧倒しないようにしてください。
"""

In [ ]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

#### マルチエージェントシステムの準備が完了しました！

## エージェントをテストしましょう。

旅行プランニングのシナリオでマルチエージェントシステムをテストしてみましょう：

In [ ]:
response = travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

In [ ]:
response = travel_agent("I would only like to focus on the flight at the moment. direct flimid-range, city center, pool, standard room")

## メモリ永続化のテスト

メモリシステムが正しく動作しているかテストするために、旅行エージェントの新しいインスタンスを作成し、以前に保存された情報にアクセスできるか確認します：

In [ ]:
# 旅行エージェントの新しいインスタンスを作成
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# 以前の会話について質問
new_travel_agent("以前話したフライトについて教えてもらえますか？")

## まとめ

このノートブックでは、以下を実演しました：

1. 複数のエージェント用の共有メモリリソースの作成方法
2. メモリアクセスを持つツールとしての専門エージェントの実装方法
3. 会話コンテキストを維持しながら複数のエージェント間を調整する方法
4. 異なるエージェントインスタンス間でメモリが永続化される方法

共有メモリを持つこのマルチエージェントアーキテクチャは、一貫したユーザーエクスペリエンスを維持しながら専門ドメインを処理できる複雑な会話 AI システムを構築するための強力なアプローチを提供します。

## クリーンアップ
このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう。

In [ ]:
#client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)